In [25]:
import sys
sys.path.append("/kaggle/input")

FIS model

In [26]:
def FIS(Turn = None,filePath='./data/Dataset/Meta_result_txl.csv',fileName=None,cluster = []):
    import numpy as np
    import pandas as pd
    from module.Rules_Function.RuleWeight import RuleWeight
    import seaborn as sns
    import matplotlib.pyplot as plt
    from module.Rules_Function.Rules_gen import rule_generate
    from module.Rules_Function.Rules_reduce import reduce_rule,remove_rule
    import pickle
    from module.Convert.var_lang import change_var_lang̣,change_var_lang̣_default
    import sys
    import time
    import os

    base_dir = "/kaggle/working/"
    input_dir = os.path.join(base_dir,f"data/FIS/input/{fileName}/")
    output_dir = os.path.join(base_dir,f"data/FIS/output/{fileName}/")
    output_dir_frb = os.path.join(base_dir,f"data/FIS/output/{fileName}/FRB/")

    if not os.path.exists(input_dir):
        os.makedirs(input_dir)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if not os.path.exists(output_dir_frb):
        os.makedirs(output_dir_frb)
    start_time = time.time()
    

        
    df = pd.read_csv(filePath)

    full_data = df

    df_full_data = pd.DataFrame(full_data)
    train_data = df_full_data.sample(frac=0.8, random_state=None)
    train_data.to_csv(os.path.join(base_dir,f'data/FIS/input/{fileName}/train_data.csv'))
    train_data = train_data.values
    test_data = df_full_data.sample(frac=0.2, random_state=None)
    # print(train_data.shape)


    full_data = np.array(full_data)
    train_data = np.array(train_data)

    min_vals = np.min(full_data, axis=0)
    max_vals = np.max(full_data, axis=0)

    min_vals_data = pd.DataFrame(min_vals)
    max_vals_data = pd.DataFrame(max_vals)
    min_vals_data.to_csv(os.path.join(base_dir,f"data/FIS/output/{fileName}/min_vals.csv"))
    max_vals_data.to_csv(os.path.join(base_dir,f"data/FIS/output/{fileName}/max_vals.csv"))
    test_data.to_csv(os.path.join(base_dir,f'data/FIS/input/{fileName}/test_data.csv'))

    h = train_data.shape[0]
    w = train_data.shape[1]
    
    lang2 = ["Low","High"]
    lang3 = ["Low","Medium","High"]
    lang5 = ["Very Low","Low","Medium","High","Very High"]

    m = 2
    esp = 0.01
    maxTest = 200

    rules,centers,U = rule_generate(h,w,train_data,cluster,min_vals,max_vals,m,esp,maxTest)

    col_num = train_data.shape[1] -1
    label = train_data[:, col_num]

    for j in range(h):
        rules[j, col_num] = np.argmax(U[j, :]) + 1

    [t, sigma_M] = RuleWeight(rules, train_data[:,:-1], cluster, centers)
    sigma_M = sigma_M.reshape(-1,1)
    sigma_M = sigma_M[:-1, :]

    sigma_M = np.hstack((sigma_M[:, [0]], sigma_M[:, [0]], sigma_M[:, [0]]))

    rules = np.hstack((rules, np.min(t, axis=1, keepdims=True), train_data[:, [col_num]]))

    df_Rule_List = pd.DataFrame(rules)
    df_Rule_List.to_csv(os.path.join(base_dir,f"data/FIS/output/{fileName}/Rule_List_All.csv"), index=False)

    rules_reduce = reduce_rule(h,col_num,rules)
    df_Rule_List1 = pd.DataFrame(rules_reduce)
    df_Rule_List1.to_csv(os.path.join(base_dir,f"data/FIS/output/{fileName}/Rule_List_reduce.csv"), index=False)

    ruleList = remove_rule(h,col_num,rules_reduce)
    ruleListLang = change_var_lang̣_default(cluster,ruleList)

    df_rule_lang = pd.DataFrame(ruleListLang)
    df_rule_lang.to_csv(os.path.join(base_dir,f"data/FIS/output/{fileName}/Rule_List_Language.csv"),index=False)
    df_rule_lang.to_csv(os.path.join(base_dir,f"data/FIS/output/{fileName}/FRB.csv"),index=False)
    
    df_rule_30 = df_rule_lang.sample(frac=0.3,random_state=42)
    df_rule_70 = df_rule_lang.sample(frac=0.7,random_state=42)
    df_rule_30.to_csv(os.path.join(base_dir,f"data/FIS/output/{fileName}/FRB/TestDataRule.csv"),index=False)
    df_rule_70.to_csv(os.path.join(base_dir,f"data/FIS/output/{fileName}/FRB/TrainDataRule.csv"),index=False)
    
    df_Rule_List = pd.DataFrame(ruleList)
    df_Rule_List.to_csv(os.path.join(base_dir,f"data/FIS/output/{fileName}/Rule_List.csv"), index=False)

    df_Sigma = pd.DataFrame(sigma_M)
    df_Sigma.to_csv(os.path.join(base_dir,f"data/FIS/output/{fileName}/Sigma_M.csv"), index=False)

    df_Centers = pd.DataFrame(centers)
    df_Centers.to_csv(os.path.join(base_dir,f"data/FIS/output/{fileName}/Centers.csv"), index=False)

    print("ruleList:",ruleList)
    print("sigma_M:", sigma_M)
    print("centers:", centers)
    model_data = {
        "ruleList": ruleList,
        "sigma_M": sigma_M,
        "centers": centers,
        "min_vals": min_vals,
        "max_vals": max_vals
    }
    totalTime = time.time() - start_time
    print("Train finish : ",totalTime)
    if not os.path.exists(os.path.join(base_dir,f"models/{fileName}/")):
        os.makedirs(os.path.join(base_dir,f"models/{fileName}/"))
    with open(os.path.join(base_dir,f"models/{fileName}/fuzzy_model.pkl"), "wb") as file:
        pickle.dump(model_data, file)
    # with open(os.path.join("/kaggle/input/",f"models/{fileName}/fuzzy_model.pkl"), "wb") as file:
    #     pickle.dump(model_data, file)
        
    FIS_Test_file(Modality = "Metadata-Image Fusion",Turn = Turn,fileName=fileName)
    totalTime = time.time() - start_time
    print("All finish : ",totalTime)

FIS Test file

In [27]:
def FIS_Test_file(Modality=None,Turn =None,fileName=None):
    import numpy as np
    from module.Membership_Function.GaussMF import GaussMF
    from models.load_model import load_model
    from module.Test.fuzzify_input import fuzzify_input
    from module.Test.match_rule import match_rule
    import pandas as pd
    from module.Test.Test import test_fis
    from sklearn.metrics import precision_score,f1_score,recall_score,accuracy_score, confusion_matrix
    import time
    import csv,os
    base_dir = "/kaggle/working/"
    startTime = time.time()
    predict_labels = []
    true_labels = []

    data = pd.read_csv(os.path.join(base_dir,f"data/FIS/input/{fileName}/test_data.csv"))
    print(os.path.join(base_dir,f"data/FIS/input/{fileName}/test_data.csv"))
    label_index = data.shape[1]-1
    for i,r in data.iterrows():
        true_labels.append(r.values[label_index])
        sample_input = r.values[1:label_index]
        predict_labels.append(test_fis(sample_input,fileName))
    true_labels = np.array(true_labels)
    predicted_labels = np.array(predict_labels)
    
    accuracy = accuracy_score(true_labels, predicted_labels)
    print(f"Độ chính xác: {accuracy:.2%}")
    precision = precision_score(true_labels, predicted_labels,average='macro')
    print(f"Độ đo precision: {precision:.2%}")
    f1 = f1_score(true_labels, predicted_labels,average='macro')
    print(f"Độ đo F1: {f1:.2%}")
    recall = recall_score(true_labels, predicted_labels,average='macro',zero_division=0)
    print(f"Độ đo Recall: {recall:.2%}")
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    print("Confusion matrix:\n")
    print(conf_matrix)
    


    
    csv_file = os.path.join(base_dir,f"data/Test/{fileName}/acc.csv")
    if(Turn!=None):
        with open(csv_file, mode="a", newline="", encoding="utf-8") as file:
            writer = csv.writer(file)
            if file.tell() == 0:
                writer.writerow(["id","Modality","models","Accuracy", "F1 Score", "Recall"])

            writer.writerow([Turn,Modality,"FIS",f"{accuracy:.2%}", f"{f1:.2%}", f"{recall:.2%}"])
    endTime = time.time() - startTime
    print("Test xong : ", endTime)
    
    results = {
        "Total Time": [endTime],
        "Test Accuracy": [accuracy],
        "Test Precision": [precision],
        "Test Recall": [recall],
        "Test F1": [f1],
    }
    dfData = pd.DataFrame(results)
    dfData.to_csv(os.path.join(base_dir,f"data/FIS/output/{fileName}/Results_FIS.csv"), index=False)


Only Table Feature PlaceMissing

In [30]:
FIS(fileName="Only Table Feature Replace Missing",
    filePath="/kaggle/input/datasetfile/OnlyTableFeatureReplaceMissing.csv",
    cluster=[2,2,5,5,3,2,2,2,2,2,2,3,5,3,3,2,3,2,3,2,2,2,6])

ruleList: [[1. 1. 3. ... 2. 2. 1.]
 [1. 1. 2. ... 1. 1. 3.]
 [1. 1. 2. ... 2. 2. 1.]
 ...
 [1. 1. 1. ... 2. 2. 1.]
 [2. 2. 2. ... 2. 2. 1.]
 [1. 1. 2. ... 1. 1. 3.]]
sigma_M: [[ 4.02616655  4.02616655  4.02616655]
 [ 3.8124749   3.8124749   3.8124749 ]
 [ 4.64231251  4.64231251  4.64231251]
 [ 4.63733994  4.63733994  4.63733994]
 [17.73736875 17.73736875 17.73736875]
 [ 3.47819036  3.47819036  3.47819036]
 [ 3.32071848  3.32071848  3.32071848]
 [ 3.35849137  3.35849137  3.35849137]
 [ 3.32963156  3.32963156  3.32963156]
 [ 3.52519878  3.52519878  3.52519878]
 [ 3.41547099  3.41547099  3.41547099]
 [ 8.42626983  8.42626983  8.42626983]
 [ 4.84370546  4.84370546  4.84370546]
 [32.83959655 32.83959655 32.83959655]
 [22.09808511 22.09808511 22.09808511]
 [ 4.25723952  4.25723952  4.25723952]
 [ 1.2739827   1.2739827   1.2739827 ]
 [ 4.37413403  4.37413403  4.37413403]
 [ 1.2739827   1.2739827   1.2739827 ]
 [ 4.29173597  4.29173597  4.29173597]
 [ 4.25025599  4.25025599  4.25025599]]
cente